In [8]:
import pandas as pd

# Load data
df = pd.read_parquet("../data/preprocessed/prices.parquet")


In [ ]:
import numpy as np

In [30]:
from sklearn.preprocessing import StandardScaler

In [11]:
print(df.columns)

MultiIndex([('Adj Close', 'BNP.PA'),
            (    'Close', 'BNP.PA'),
            (     'High', 'BNP.PA'),
            (      'Low', 'BNP.PA'),
            (     'Open', 'BNP.PA'),
            (   'Volume', 'BNP.PA'),
            ('Adj Close',  'ML.PA'),
            (    'Close',  'ML.PA'),
            (     'High',  'ML.PA'),
            (      'Low',  'ML.PA'),
            ...
            (     'High',  'SU.PA'),
            (      'Low',  'SU.PA'),
            (     'Open',  'SU.PA'),
            (   'Volume',  'SU.PA'),
            ('Adj Close', 'VIV.PA'),
            (    'Close', 'VIV.PA'),
            (     'High', 'VIV.PA'),
            (      'Low', 'VIV.PA'),
            (     'Open', 'VIV.PA'),
            (   'Volume', 'VIV.PA')],
           names=['Price', 'Ticker'], length=162)


In [17]:
adj_close = df.xs('Adj Close', axis=1, level='Price')

# Compute log returns
log_returns = np.log(adj_close / adj_close.shift(1))

# Create MultiIndex for log returns
log_returns.columns = pd.MultiIndex.from_product(
    [['LogReturn'], log_returns.columns],
    names=['Price', 'Ticker']
)

# Concatenate and sort
df_combined = pd.concat([df, log_returns], axis=1).sort_index(axis=1)

In [ ]:
print(df_combined.columns)


MultiIndex([('Adj Close', 'ACA.PA'),
            ('Adj Close',  'AI.PA'),
            ('Adj Close', 'AIR.PA'),
            ('Adj Close', 'ATO.PA'),
            ('Adj Close', 'BNP.PA'),
            ('Adj Close',  'CA.PA'),
            ('Adj Close', 'CAP.PA'),
            ('Adj Close',  'DG.PA'),
            ('Adj Close', 'DSY.PA'),
            ('Adj Close',  'EL.PA'),
            ...
            (   'Volume',  'ML.PA'),
            (   'Volume',  'OR.PA'),
            (   'Volume',  'RI.PA'),
            (   'Volume', 'RNO.PA'),
            (   'Volume', 'SAN.PA'),
            (   'Volume',  'SU.PA'),
            (   'Volume',  'SW.PA'),
            (   'Volume', 'VIE.PA'),
            (   'Volume', 'VIV.PA'),
            (   'Volume', 'WLN.PA')],
           names=['Price', 'Ticker'], length=189)


In [ ]:
print(df_combined.head)

In [ ]:
nan_summary = df_combined.isnull().sum()
nan_summary = nan_summary[nan_summary > 0] 
print(nan_summary)

In [23]:
tickers = df_combined.columns.get_level_values("Ticker").unique()

In [31]:
features_to_keep = ["Adj Close", "Volume", "LogReturn"]
cols_to_keep = [col for col in df_combined.columns if col[0] in features_to_keep]
df_filtered = df_combined[cols_to_keep].copy()

In [32]:
normalized_dfs = []
for feature in ["Adj Close", "Volume"]:
    feature_data = df_filtered.xs(feature, axis=1, level='Price')
    normalized_cols = []
    
    for ticker in tickers:
        series = feature_data[ticker].copy()
        scaler = StandardScaler()
        normalized_values = scaler.fit_transform(series.values.reshape(-1, 1)).flatten()
        normalized_cols.append(pd.Series(normalized_values, index=series.index, name=ticker))
    
    normalized_df = pd.concat(normalized_cols, axis=1)
    normalized_df.columns = pd.MultiIndex.from_product([[feature], normalized_df.columns], names=['Price', 'Ticker'])
    normalized_dfs.append(normalized_df)

# Keep LogReturn as-is
logreturns = df_filtered.xs('LogReturn', axis=1, level='Price')
logreturns.columns = pd.MultiIndex.from_product([['LogReturn'], logreturns.columns], names=['Price', 'Ticker'])
normalized_dfs.append(logreturns)

# Combine all features
data_ppo = pd.concat(normalized_dfs, axis=1).sort_index(axis=1)


In [ ]:
data_ppo = data_ppo.dropna()

After removing NaN: (1792, 81)


In [36]:
def calculate_rsi(prices, window=14):
    """Calculate RSI indicator"""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_macd(prices, fast=12, slow=26, signal=9):
    """Calculate MACD indicator"""
    ema_fast = prices.ewm(span=fast).mean()
    ema_slow = prices.ewm(span=slow).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal).mean()
    macd_histogram = macd_line - signal_line
    return macd_line, signal_line, macd_histogram

def calculate_bollinger_bands(prices, window=20, num_std=2):
    """Calculate Bollinger Bands"""
    rolling_mean = prices.rolling(window=window).mean()
    rolling_std = prices.rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)
    bb_position = (prices - lower_band) / (upper_band - lower_band)  # 0-1 normalized position
    return bb_position

def calculate_rolling_volatility(returns, window=20):
    """Calculate rolling volatility from returns"""
    return returns.rolling(window=window).std() * np.sqrt(252)  # Annualized

def calculate_volume_sma_ratio(volume, window=20):
    """Volume relative to its moving average"""
    volume_sma = volume.rolling(window=window).mean()
    return volume / volume_sma

# Extract price and volume data for indicator calculation
adj_close = data_ppo.xs('Adj Close', axis=1, level='Price')  # This is already normalized
volume = data_ppo.xs('Volume', axis=1, level='Price')      # This is already normalized
logreturns = data_ppo.xs('LogReturn', axis=1, level='Price')

print("Calculating technical indicators...")

# We need original prices for some indicators, so let's get them
# Load original data for indicator calculation
df_original = df_combined
adj_close_orig = df_original.xs('Adj Close', axis=1, level='Price')
volume_orig = df_original.xs('Volume', axis=1, level='Price')


Calculating technical indicators...


In [37]:
indicator_dfs = []
tickers = adj_close.columns

# 1. RSI (momentum)
print("Calculating RSI...")
rsi_data = []
for ticker in tickers:
    rsi = calculate_rsi(adj_close_orig[ticker])
    # Normalize RSI to 0-1 range (RSI is naturally 0-100)
    rsi_normalized = (rsi - 50) / 50  # Center around 0, range roughly -1 to 1
    rsi_data.append(pd.Series(rsi_normalized, index=adj_close.index, name=ticker))

rsi_df = pd.concat(rsi_data, axis=1)
rsi_df.columns = pd.MultiIndex.from_product([['RSI'], rsi_df.columns], names=['Price', 'Ticker'])
indicator_dfs.append(rsi_df)

# 2. MACD (trend and momentum)
print("Calculating MACD...")
macd_data = []
for ticker in tickers:
    macd_line, signal_line, macd_histogram = calculate_macd(adj_close_orig[ticker])
    # Use MACD histogram (already oscillates around 0)
    # Normalize by rolling standard deviation
    macd_norm = macd_histogram / macd_histogram.rolling(window=50).std()
    macd_data.append(pd.Series(macd_norm, index=adj_close.index, name=ticker))

macd_df = pd.concat(macd_data, axis=1)
macd_df.columns = pd.MultiIndex.from_product([['MACD'], macd_df.columns], names=['Price', 'Ticker'])
indicator_dfs.append(macd_df)

# 3. Bollinger Band Position (mean reversion)
print("Calculating Bollinger Bands...")
bb_data = []
for ticker in tickers:
    bb_position = calculate_bollinger_bands(adj_close_orig[ticker])
    # BB position is already 0-1, center it around 0
    bb_centered = (bb_position - 0.5) * 2  # Now -1 to 1 range
    bb_data.append(pd.Series(bb_centered, index=adj_close.index, name=ticker))

bb_df = pd.concat(bb_data, axis=1)
bb_df.columns = pd.MultiIndex.from_product([['BB_Position'], bb_df.columns], names=['Price', 'Ticker'])
indicator_dfs.append(bb_df)

# 4. Rolling Volatility (risk measure)
print("Calculating Rolling Volatility...")
vol_data = []
for ticker in tickers:
    rolling_vol = calculate_rolling_volatility(logreturns[ticker])
    # Normalize volatility using StandardScaler
    scaler = StandardScaler()
    vol_norm = scaler.fit_transform(rolling_vol.dropna().values.reshape(-1, 1)).flatten()
    vol_series = pd.Series(index=rolling_vol.index, dtype=float)
    vol_series.loc[rolling_vol.dropna().index] = vol_norm
    vol_data.append(pd.Series(vol_series, index=adj_close.index, name=ticker))

vol_df = pd.concat(vol_data, axis=1)
vol_df.columns = pd.MultiIndex.from_product([['Volatility'], vol_df.columns], names=['Price', 'Ticker'])
indicator_dfs.append(vol_df)

# 5. Volume Ratio (volume momentum)
print("Calculating Volume Ratio...")
vol_ratio_data = []
for ticker in tickers:
    vol_ratio = calculate_volume_sma_ratio(volume_orig[ticker])
    # Log transform and normalize
    vol_ratio_log = np.log(vol_ratio.clip(lower=0.1))  # Avoid log(0)
    scaler = StandardScaler()
    vol_ratio_norm = scaler.fit_transform(vol_ratio_log.dropna().values.reshape(-1, 1)).flatten()
    vol_ratio_series = pd.Series(index=vol_ratio_log.index, dtype=float)
    vol_ratio_series.loc[vol_ratio_log.dropna().index] = vol_ratio_norm
    vol_ratio_data.append(pd.Series(vol_ratio_series, index=adj_close.index, name=ticker))

vol_ratio_df = pd.concat(vol_ratio_data, axis=1)
vol_ratio_df.columns = pd.MultiIndex.from_product([['Volume_Ratio'], vol_ratio_df.columns], names=['Price', 'Ticker'])
indicator_dfs.append(vol_ratio_df)

# Combine original features with new indicators
print("Combining all features...")
all_features = [data_ppo] + indicator_dfs
data_ppo_enhanced = pd.concat(all_features, axis=1).sort_index(axis=1)

Calculating RSI...
Calculating MACD...
Calculating Bollinger Bands...
Calculating Rolling Volatility...
Calculating Volume Ratio...
Combining all features...


In [38]:
data_ppo_enhanced = data_ppo_enhanced.dropna()
print(f"After removing NaN: {data_ppo_enhanced.shape}")

# Check the structure
print("\nFeature summary:")
feature_names = data_ppo_enhanced.columns.get_level_values('Price').unique()
print(f"Features: {list(feature_names)}")
print(f"Number of tickers: {len(tickers)}")
print(f"Total columns: {len(data_ppo_enhanced.columns)}")

# Verify all features are properly normalized (should be roughly mean=0, std=1)
print("\nFeature statistics (first 5 columns):")
print(data_ppo_enhanced.iloc[:, :5].describe())

# Check for any remaining issues
print(f"\nData quality check:")
print(f"Any NaN values: {data_ppo_enhanced.isnull().sum().sum()}")
print(f"Any infinite values: {np.isinf(data_ppo_enhanced.select_dtypes(include=[np.number])).sum().sum()}")
print(f"Date range: {data_ppo_enhanced.index.min()} to {data_ppo_enhanced.index.max()}")


After removing NaN: (1744, 216)

Feature summary:
Features: ['Adj Close', 'BB_Position', 'LogReturn', 'MACD', 'RSI', 'Volatility', 'Volume', 'Volume_Ratio']
Number of tickers: 27
Total columns: 216

Feature statistics (first 5 columns):
Price     Adj Close                                                    
Ticker       ACA.PA        AI.PA       AIR.PA       ATO.PA       BNP.PA
count   1744.000000  1744.000000  1744.000000  1744.000000  1744.000000
mean       0.000355     0.039857     0.026985    -0.044309     0.004629
std        1.013749     0.985065     1.000512     0.977684     1.013735
min       -2.115286    -1.552469    -2.519697    -1.399553    -2.078222
25%       -0.639548    -0.773973    -0.600832    -1.060883    -0.850501
50%       -0.218137    -0.069115     0.031741     0.073531    -0.095871
75%        0.467950     0.857378     0.732689     0.834278     0.880171
max        2.653312     2.064923     2.539568     3.021541     2.234117

Data quality check:
Any NaN values: 0
Any 

In [ ]:
print(f"\n--- PPO FORMAT CHECK ---")
feature_order = ['Adj Close', 'BB_Position', 'LogReturn', 'MACD', 'RSI', 'Volatility', 'Volume', 'Volume_Ratio']
ticker_order = sorted(tickers)

# Show what the flattened column names would look like
flat_columns = []
for feature in feature_order:
    for ticker in ticker_order:
        flat_columns.append(f"{feature}_{ticker}")


--- PPO FORMAT CHECK ---
Flattened format would have 216 features per timestep
Example column names: ['Adj Close_ACA.PA', 'Adj Close_AI.PA', 'Adj Close_AIR.PA', 'Adj Close_ATO.PA', 'Adj Close_BNP.PA', 'Adj Close_CA.PA', 'Adj Close_CAP.PA', 'Adj Close_DG.PA', 'Adj Close_DSY.PA', 'Adj Close_EL.PA']...
Ready for PPO: ✅

Sample data (first 3 rows, first 10 columns):
Price      Adj Close                                                    \
Ticker        ACA.PA     AI.PA    AIR.PA    ATO.PA    BNP.PA     CA.PA   
Date                                                                     
2018-03-12 -0.231699 -1.460016 -0.776506  1.400818 -0.280430 -0.146494   
2018-03-13 -0.252785 -1.483130 -0.784590  1.346474 -0.295871 -0.249795   
2018-03-14 -0.296365 -1.495737 -0.800373  1.372411 -0.344767 -0.373755   

Price                                               
Ticker        CAP.PA     DG.PA    DSY.PA     EL.PA  
Date                                                
2018-03-12 -1.104773 -1.251393

In [41]:
def flatten_multiindex_data(df, feature_order, ticker_order):
    """Efficiently flatten MultiIndex DataFrame for PPO"""
    flattened_data = []
    
    for feature in feature_order:
        try:
            feature_data = df.xs(feature, axis=1, level='Price')
            for ticker in ticker_order:
                flattened_data.append(feature_data[ticker].values)
        except KeyError:
            print(f"Warning: Feature {feature} not found")
            # Add zeros if feature missing
            for ticker in ticker_order:
                flattened_data.append(np.zeros(len(df)))
    
    return np.column_stack(flattened_data)

# Create flattened array
flattened_array = flatten_multiindex_data(data_ppo_enhanced, feature_order, ticker_order)

# Create final PPO DataFrame
data_ppo_flat = pd.DataFrame(
    flattened_array, 
    index=data_ppo_enhanced.index, 
    columns=flat_columns
)

print(f"Final PPO data shape: {data_ppo_flat.shape}")
print(f"Date range: {data_ppo_flat.index.min()} to {data_ppo_flat.index.max()}")

# Quality check
print(f"\nFinal quality check:")
print(f"Any NaN values: {data_ppo_flat.isnull().sum().sum()}")
print(f"Any infinite values: {np.isinf(data_ppo_flat).sum().sum()}")

# Show sample
print(f"\nSample PPO data (first 3 rows, first 10 columns):")
print(data_ppo_flat.iloc[:3, :10])
print(f"\nColumn names: {list(data_ppo_flat.columns[:10])}...")

Final PPO data shape: (1744, 216)
Date range: 2018-03-12 00:00:00 to 2024-12-30 00:00:00

Final quality check:
Any NaN values: 0
Any infinite values: 0

Sample PPO data (first 3 rows, first 10 columns):
            Adj Close_ACA.PA  Adj Close_AI.PA  Adj Close_AIR.PA  \
Date                                                              
2018-03-12         -0.231699        -1.460016         -0.776506   
2018-03-13         -0.252785        -1.483130         -0.784590   
2018-03-14         -0.296365        -1.495737         -0.800373   

            Adj Close_ATO.PA  Adj Close_BNP.PA  Adj Close_CA.PA  \
Date                                                              
2018-03-12          1.400818         -0.280430        -0.146494   
2018-03-13          1.346474         -0.295871        -0.249795   
2018-03-14          1.372411         -0.344767        -0.373755   

            Adj Close_CAP.PA  Adj Close_DG.PA  Adj Close_DSY.PA  \
Date                                                      

In [43]:
# Save the flattened PPO-ready data
data_ppo_flat.to_parquet("../data/preprocessed/data_ppo.parquet")
print("✅ Saved PPO-ready data to data_ppo.parquet")

✅ Saved PPO-ready data to data_ppo.parquet
